In [1]:
!pip install kafka-python
!pip install redis

In [2]:
import os
import requests
import json
import redis
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from datetime import datetime

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[2] pyspark-shell'
conf = SparkConf().set("spark.jars","/home/jovyan/work/spark-streaming-kafka-0-8-assembly_2.11-2.4.4.jar")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,60) # 60 is the batch interval :)
debug = True
saveToLocalDisk = True
spark = SparkSession(sc)

In [4]:
def SaveToNewsFile(rdd):
    if (not rdd.isEmpty()):
        rdd.toDF() \
        .write.save("/home/jovyan/work/ClickData.csv",
                    maxRecordsPerFile=1000, format="csv", mode="append") 

In [ ]:
kafkaStream = KafkaUtils.createStream(
    ssc=ssc, 
    zkQuorum='zk-cs:2181', 
    groupId='test-consumer-group',#, #Consumer Group 
    topics={'click':1})

olines = kafkaStream.map(lambda x: x[1]).map(lambda a: json.loads(a))
if(debug):
    olines.pprint()
if(saveToLocalDisk):
    lines = olines.filter(lambda a: ('type' in a) & (a['type']=="click")) \
    .map(lambda a: (a['user'],a['article'])) \
    .foreachRDD(SaveToNewsFile)
    
ssc.start()  
print("Going to wait termination")
ssc.awaitTermination()

Going to wait termination
-------------------------------------------
Time: 2019-12-09 05:53:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:54:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:55:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:56:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:57:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:58:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:59:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 06:00:00
-------------------------------------------

-------------------------------------------
Time: 2019